In [199]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime, timezone
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import shapely as shp
from shapely.geometry import Point, LineString
import plotly.express as px

import folium
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap

plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (20,9)

#pd.options.display.float_format = '{:.2f}'.format;

In [200]:
import os, sys
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 0.0 Get External Data

### ny_counties, social_2019_unstack, age_2019

* 2019 reported census data for **`62 Counties`** in **`New York`** (by County)
* QGIS exported geometry_multipoligon (by County)
* we will combine all via common `IBRC_Geo_ID` (which prepresents County)

# 0.1 Geo Data

In [201]:
ny_counties = gpd.read_file('/kaggle/input/ny-geo-data/newyork-counties.geojson')
ny_counties['IBRC_Geo_ID'] = ny_counties['geoid'].str[-5:]
ny_counties['state_geo_ID'] = ny_counties['geoid'].str[-5:-3]
ny_counties['county_geo_ID'] = ny_counties['geoid'].str[-3:]
ny_counties['IBRC_Geo_ID']=ny_counties['IBRC_Geo_ID'].astype('int')
ny_counties

# 0.2 Social Context Data (pre-COVID: 1972-2019)

In [202]:
# import csv
social = pd.read_csv('/kaggle/input/ny-social-context/Social Context.csv')
social = social[social.Description.str.contains(', NY')]
social

# reported in 2019, mostly 2017
social['Time_Period'].str[-4:].value_counts()
print(f'Reported in 2019, but {1178/(1178+62)*100}% comes from 2017 survey data.')
print(f'The rest 5% comes from {social["Time_Period"].str[:4].value_counts().index[0]} to 2013 survey data.')
print('\n')

# drop cols
social[['IBRC_Geo_ID','Year']].groupby('IBRC_Geo_ID').max().describe()
social_2019 = social[social.Year == 2019]
social_2019 = social_2019.reset_index().drop(columns={'index'})
social_2019 = social_2019.iloc[:, ([0, 3, 4, 5, 6, 7])].reset_index()
social_2019

In [203]:
social_2019.IBRC_Geo_ID.nunique()

In [204]:
# social code dictionary for mapping
social_code_df =  social_2019[['Social_Context_Code','Social_Context_Code_Description']].groupby('Social_Context_Code_Description').mean().astype(int).unstack().to_frame().reset_index().drop(columns='level_0').rename(columns={0:'Social_Context_Code'})
social_code_dict = social_code_df.set_index('Social_Context_Code').to_dict()
social_code_dict

In [205]:
social_2019_pivot = pd.pivot_table(social_2019, values='Social_Context_Domain_Data', index=['IBRC_Geo_ID','Description','Social_Context_Code_Description'], aggfunc='median')
social_2019_unstack = social_2019_pivot.unstack()
social_2019_unstack = social_2019_unstack.droplevel(level=0, axis=1)
social_2019_unstack

In [206]:

social_2019.Social_Context_Code_Description.value_counts()

# 0.3 Age Data

In [207]:
age = pd.read_csv('/kaggle/input/ny-social-context/Population by Age and Sex - US States Counties.csv')
age = age[age.Description.str.contains(', NY')]
age[['IBRC_Geo_ID','Year']].groupby('IBRC_Geo_ID').max().describe()
age_2019 = age[age.Year == 2019]
age_2019 = age_2019.reset_index().drop(columns=['index','Statefips','Countyfips','Year','Male Population', 'Female Population'])
age_2019

In [208]:
age_2019['percent_Population 0-4'] = age_2019['Population 0-4'] / age_2019['Total Population']
age_2019['percent_Population 5-17'] = age_2019['Population 5-17'] / age_2019['Total Population']
age_2019['percent_Population 18-24'] = age_2019['Population 18-24'] / age_2019['Total Population']
age_2019['percent_Population 25-44'] = age_2019['Population 25-44'] / age_2019['Total Population']
age_2019['percent_Population 45-64'] = age_2019['Population 45-64'] / age_2019['Total Population']
age_2019['percent_Population 65+'] = age_2019['Population 65+'] / age_2019['Total Population']
age_2019

In [209]:
age_2019.IBRC_Geo_ID.nunique()

# 0.4 Tree Data

### New York Tree Equity

In [210]:
ny_tree_equity = gpd.read_file('/kaggle/input/treeequityscore/ny.shp')
ny_tree_equity['IBRC_Geo_ID'] = ny_tree_equity['geoid'].astype(str).str[:5]
ny_tree_equity.head(1).T;

In [211]:
# treecanopy = -1 for all values. we can drop this.
ny_tree_equity['treecanopy'].value_counts()

### convert column types

In [212]:
int_cols = ['geoid', 'IBRC_Geo_ID']

str_cols = ['state', 'county','geometry', 'ua_name', 'incorpname', 'congressio', 'biome','source']

float_cols = ['total_pop', 'pctpov', 'pctpoc',
       'unemplrate', 'medhhinc', 'dep_ratio', 'child_perc', 'seniorperc', 
       'treecanopy', 'area', 'avg_temp', 'bgpopdense', 'popadjust', 'tc_gap', 'tc_goal',
       'phys_hlth', 'ment_hlth', 'asthma', 'core_m', 'core_w', 'core_norm',
       'healthnorm', 'priority', 'tes', 'tesctyscor']

# geo_cols = ['geometry']

In [213]:
ny_tree_equity[int_cols] = ny_tree_equity[int_cols].astype(int)

ny_tree_equity[str_cols] = ny_tree_equity[str_cols].astype(str)

ny_tree_equity[float_cols] = ny_tree_equity[float_cols].astype(float)

ny_tree_equity['geoid_county'] = ny_tree_equity['geoid'].astype(str).str[:5]

ny_tree_equity.head(1).T;

In [214]:
tmp = ny_tree_equity[['geoid','geoid_county']].groupby('geoid_county').median()
print('number of counties in TREE data', tmp.shape)

tmp1 = tmp.geoid.astype(int).to_frame()
geoid_geoid_county_dict = tmp1.copy()
geoid_geoid_county_dict = geoid_geoid_county_dict.reset_index()
geoid_geoid_county_dict.head()

In [215]:
ny_tree_equity_county_num = ny_tree_equity.groupby('county').mean()
ny_tree_equity_county_num.head(1).T;

In [216]:
#pd.options.display.float_format = '{:.0f}'.format;
pd.merge(ny_tree_equity_county_num, geoid_geoid_county_dict, on='geoid', how='left');


In [217]:
ny_tree_equity_county_num.isnull().sum()

In [218]:
column_median = ny_tree_equity_county_num.median()
column_median

In [219]:
# fill missing values with median
ny_tree_equity_county_num = ny_tree_equity_county_num.fillna(column_median)

# we will merge only selected cols from TREE data to Q 
ny_tree_equity_county_num.columns.to_list();

selected_cols = ['pctpov', 'pctpoc', 'unemplrate', 'medhhinc',
       'dep_ratio','treecanopy', 
       'avg_temp', 'phys_hlth',
       'ment_hlth', 'asthma', 'core_norm', 'healthnorm',
       'tes', 'tesctyscor',]

ny_tree_equity_county_all = ny_tree_equity_county_num[selected_cols].rename(columns={'pctpov':'percent_poverty',
 'pctpoc':'percent_people_of_color',
 'unemplrate':'unemployment_rate',
 'medhhinc':'median_household_income',
 'dep_ratio': 'dependency_ratio',
 'treecanopy':'tree_canopy',
 'avg_temp':'hot_summer_avg_temp',
 'phys_hlth':'physical_health_challenges',
 'ment_hlth':'mental_health_challenges',
 'asthma':'athma_challenge',
 'core_norm':'heart challenges',
 'healthnorm':'health_problem',
 'tes':'tree_equity_score',
 'tesctyscor':'tree_equity_score_municipal'})

ny_tree_equity_county_all[['tree_canopy']]

In [220]:
# add 'geoid' column here (geoid:geoid_county)
ny_tree_equity_county_all['geoid_county'] = ny_tree_equity_county_num['geoid'].astype(str).str[:5]
ny_tree_equity_county_all.drop(columns=['tree_canopy','tree_equity_score_municipal'], inplace=True)
ny_tree_equity_county_all

# 0.5 Merge NY_Geo (by 'Geoid_county')

In [221]:
# ny_counties, social_2019_unstack, age_2019

ny_geo = pd.merge(ny_counties, social_2019_unstack, on='IBRC_Geo_ID', how='left')
ny_geo = pd.merge(ny_geo, age_2019, on='IBRC_Geo_ID', how='left')
ny_geo['geoid_county'] = ny_geo['geoid'].astype(str).str[-5:]

# this is important TREES vs QGIS
ny_geo = pd.merge(ny_geo, ny_tree_equity_county_all, on='geoid_county', how='left')

#np_geo = pd.merge(ny_geo, epa_ny_counties, on='name', how='left')
ny_geo.to_csv('./ny-counties-1_cols_added_v1.csv')
ny_geo

In [222]:
ny_geo.isnull().sum()

## Counties with TREE data

In [223]:
print('Counties with TREE data:')
print(ny_geo[ny_geo['percent_poverty'].notna()].shape[0])
ny_geo[ny_geo['percent_poverty'].notna()]['name']

## Counties w/ missing TREE data

In [224]:
print('Counties with missing TREE data: ')
print(ny_geo[ny_geo['percent_poverty'].isna()].shape[0])
ny_geo[ny_geo['percent_poverty'].isna()]['name']

In [225]:
# missing values in final ny_geo dataframe
pd.DataFrame(ny_geo.isnull().sum()).sort_values(0,ascending=False).style.background_gradient(axis=0)

### Fill in missing data (median)

In [226]:
# median values
median_vals = ny_geo.median()
print('median values of each column: \n\n', median_vals);

In [227]:
# fill in missing values with median of each column
ny_geo_final = ny_geo.fillna(median_vals)
ny_geo_final['athma_challenge'].value_counts()

In [228]:
# missing values in final ny_geo dataframe
pd.DataFrame(ny_geo_final.isnull().sum()).sort_values(0,ascending=False).style.background_gradient(axis=0)

In [229]:
corr = ny_geo_final.corr()

mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(35, 20))
    ax = sns.heatmap(corr, mask=mask, square=False, annot=True, fmt='.1f', annot_kws={"fontsize":15})
sns.set(font_scale=1.4)

In [230]:
ny_geo_final.describe().T

## Export to CSV

In [231]:
print(ny_geo_final.shape)
ny_geo_final.to_csv('./ny_geo_final.csv')

In [232]:
pd.read_csv('/kaggle/input/ny-final/ny_geo_final-2.csv')

In [233]:
ny_geo_final.head(1).T

In [234]:
display(ny_geo_final.columns.tolist())

In [235]:
ny_geo_final.describe()

## Export to JSON

In [236]:
#pd.read_json((jDict['data']).to_json(), orient='index')
#df.to_json(r'Path to store the exported JSON file\File Name.json')
#df.to_json(orient='index')
ny_geo_final.to_json();

## Radar Chart

In [237]:
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [238]:
ny_geo_final.dtypes
num_cols = ny_geo_final.select_dtypes(include=np.number).columns.tolist()
ny_geo_final[num_cols]


In [239]:
# New York County (index = 29) Example
scaler = MinMaxScaler()
print(scaler.fit(ny_geo_final[num_cols]))
print('max values: \n')
print(scaler.data_max_)
print('\ntransformed within new max values:\n')
print(scaler.transform(ny_geo_final[num_cols])[29])

In [240]:
scaler = MinMaxScaler()
scaled_ny = scaler.fit_transform(ny_geo_final[num_cols])
scaled_ny = pd.DataFrame(scaled_ny, columns=num_cols)
scaled_ny.head(1).T

In [241]:
ny_geo_final[ny_geo_final['name'] == 'New York County']

In [242]:
scaled_ny.iloc[29]

In [243]:
scaled_ny[['heart challenges','athma_challenge','Entrepreneurship','tree_equity_score','Hopefulness','mental_health_challenges','Gender Equality','hot_summer_avg_temp','Belief In Science','Neuroticism','Income Per Capita','Income Mobility']].loc[29].values

In [244]:
df = scaled_ny[['heart challenges','athma_challenge','Entrepreneurship','tree_equity_score','Hopefulness','mental_health_challenges','Gender Equality','hot_summer_avg_temp','Belief In Science','Neuroticism','Income Per Capita','Income Mobility']]
cols = scaled_ny[['heart challenges','athma_challenge','Entrepreneurship','tree_equity_score','Hopefulness','mental_health_challenges','Gender Equality','hot_summer_avg_temp','Belief In Science','Neuroticism','Income Per Capita','Income Mobility']].columns


fig = px.line_polar(
                    r=df.loc[29].values,
                    theta=cols,
                    line_close=True,
                    range_r = [0,1.0],
                    title='New York County vs Other Counties in New York State:')
fig.update_traces(fill='toself')
fig.show()

## Radar Chart - Bronx County

Where in New York do you want to move to next?
* large trunk trees (>20in) (distance to large trees) (county levvel large tree count)
* clean air (low athma, low lung desease, low Ozone, low PM2.5)
* better physical health (low heart desease (low PM 2.5), low lung desease (low Ozone)
* clean water 
* close to water body (distance to water body)

In [245]:
ny_geo_final[ny_geo_final['name'] == 'Bronx County']

In [246]:
scaled_ny[['heart challenges','athma_challenge','Entrepreneurship','tree_equity_score','Hopefulness','mental_health_challenges','Gender Equality','hot_summer_avg_temp','Belief In Science','Neuroticism','Income Per Capita','Income Mobility']].loc[42].values

In [247]:
df = scaled_ny[['heart challenges','athma_challenge','Entrepreneurship','tree_equity_score','Hopefulness','mental_health_challenges','Gender Equality','hot_summer_avg_temp','Belief In Science','Neuroticism','Income Per Capita','Income Mobility']]
cols = scaled_ny[['heart challenges','athma_challenge','Entrepreneurship','tree_equity_score','Hopefulness','mental_health_challenges','Gender Equality','hot_summer_avg_temp','Belief In Science','Neuroticism','Income Per Capita','Income Mobility']].columns


fig = px.line_polar(
                    r=df.loc[42].values,
                    theta=cols,
                    line_close=True,
                    range_r = [0,1.0],
                    title='Bronx County vs Other Counties in New York State:')
fig.update_traces(fill='toself')
fig.show()

## Radar Chart - Niagara County

In [248]:
ny_geo_final[ny_geo_final['name'] == 'Niagara County']

In [249]:
scaled_ny[['heart challenges','athma_challenge','Entrepreneurship','tree_equity_score','Hopefulness','mental_health_challenges','Gender Equality','hot_summer_avg_temp','Belief In Science','Neuroticism','Income Per Capita','Income Mobility']].loc[60].values

In [250]:
df = scaled_ny[['heart challenges','athma_challenge','Entrepreneurship','tree_equity_score','Hopefulness','mental_health_challenges','Gender Equality','hot_summer_avg_temp','Belief In Science','Neuroticism','Income Per Capita','Income Mobility']]
cols = scaled_ny[['heart challenges','athma_challenge','Entrepreneurship','tree_equity_score','Hopefulness','mental_health_challenges','Gender Equality','hot_summer_avg_temp','Belief In Science','Neuroticism','Income Per Capita','Income Mobility']].columns


fig = px.line_polar(
                    r=df.loc[60].values,
                    theta=cols,
                    line_close=True,
                    range_r = [0,1.0],
                    title='Niagara County vs Other Counties in New York State:')
fig.update_traces(fill='toself')
fig.show()

## Radar Char - Orange County

In [251]:
ny_geo_final[ny_geo_final['name'] == 'Orange County']

In [252]:
scaled_ny[['heart challenges','athma_challenge','Entrepreneurship','tree_equity_score','Hopefulness','mental_health_challenges','Gender Equality','hot_summer_avg_temp','Belief In Science','Neuroticism','Income Per Capita','Income Mobility']].loc[54].values

In [253]:
df = scaled_ny[['heart challenges','athma_challenge','Entrepreneurship','tree_equity_score','Hopefulness','mental_health_challenges','Gender Equality','hot_summer_avg_temp','Belief In Science','Neuroticism','Income Per Capita','Income Mobility']]
cols = scaled_ny[['heart challenges','athma_challenge','Entrepreneurship','tree_equity_score','Hopefulness','mental_health_challenges','Gender Equality','hot_summer_avg_temp','Belief In Science','Neuroticism','Income Per Capita','Income Mobility']].columns


fig = px.line_polar(
                    r=df.loc[54].values,
                    theta=cols,
                    line_close=True,
                    range_r = [0,1.0],
                    title='Orange County vs Other Counties in New York State:')
fig.update_traces(fill='toself')
fig.show()